In [15]:
import gym
import time
import torch as T

In [3]:
env = gym.make('MsPacman-v0')

In [10]:
env.action_space.sample()

4

In [8]:
env.observation_space.sample().shape

(210, 160, 3)

In [19]:
done = False
s = env.reset()

while not done:
    env.render()
    action = env.action_space.sample()
    s_, r, done, _ = env.step(action)
    time.sleep(1/60)
    
env.close()